In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv('/content/drive/My Drive/dataset/ttran.csv')

In [ ]:
data.drop('id',axis=1,inplace=True)

In [ ]:
llabel=[]
llabel1=[]
llabel2=[]
llabel3=[]
llabel4=[]
llabel5=[]

In [ ]:
llabel=list(data['toxic'])
llabel1=list(data['severe_toxic'])
llabel2=list(data['obscene'])
llabel3=list(data['threat'])
llabel4=list(data['insult'])
llabel5=list(data['identity_hate'])

In [ ]:
labell=[]

In [ ]:
for a,b,c,d,e,f in zip(llabel,llabel1,llabel2,llabel3,llabel4,llabel5):
  labell.append([a,b,c,d,e,f])

In [ ]:
import tensorflow as tf

In [ ]:
len(labell)

159571

In [ ]:
import re
def preprocess_sentence(w):
  w = w.lower()
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
  w = w.strip()
  w = '<start> ' + w + ' <end>'
  return w

In [ ]:
texts=[]
a=0

In [ ]:
for line in data['comment_text']:
  a=preprocess_sentence(line)
  texts.append(a)

In [ ]:
! pip install transformers

     |████████████████████████████████| 778kB 3.2MB/s 
     |████████████████████████████████| 3.0MB 8.7MB/s 
     |████████████████████████████████| 890kB 32.6MB/s 
     |████████████████████████████████| 1.1MB 38.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=e89a88046be15e2acf0ace1f7f1458521eb02646effe87f1f61a232d829a3897
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import BertTokenizer,TFBertForSequenceClassification

In [ ]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
inputs=tokenizer(texts,return_tensors="tf",padding=True,truncation=True,max_length=200)

In [ ]:
dataset=tf.data.Dataset.from_tensor_slices((inputs['input_ids'],labell))

In [ ]:
next(iter(dataset))

(<tf.Tensor: shape=(200,), dtype=int32, numpy=
 array([  101,  1026,  2707,  1028,  7526,  2339,  1996, 10086,  2015,
         2081,  2104,  2026,  5310, 18442, 13076, 12392,  2050,  5470,
         2020, 16407,  1029,  2027,  4694,  1056,  3158,  9305, 22556,
         1010,  2074,  8503,  2006,  2070,  3806,  2044,  1045,  5444,
         2012,  2047,  2259, 14421,  6904,  2278,  1012,  1998,  3531,
         2123,  1056,  6366,  1996, 23561,  2013,  1996,  2831,  3931,
         2144,  1045,  1049,  3394,  2085,  1012,  1012,  1012,  1012,
         1026,  2203,  1028,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,  

In [ ]:
dataset1=dataset.shuffle(15000).batch(8)

In [ ]:
from keras import models,layers

In [ ]:
model=TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=6)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['dropout_76', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
 model.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(dataset.shuffle(1000).batch(16), epochs=3, batch_size=16)